# Setup Variables

MNIST, FashionMNIST, GTSRB

In [ ]:
DATASET = 'GTSRB'
SEED = 42
CUDA = 0
GPU_NAME = f'cuda:{CUDA}'

In [ ]:
import os
from pathlib import Path

base = Path().cwd()

if base.name != 'runtime-monitoring':
    os.chdir('../')
    base = Path().cwd()

# Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from fastprogress import progress_bar, master_bar

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
from torchinfo import summary


cudnn.benchmark = True
torch.set_float32_matmul_precision('high')

In [ ]:
from utilities.utils import *
from utilities.pathManager import fetchPaths
from utilities.scaleFunctions import *
from utilities.pcaFunctions import *

In [ ]:
# disable warnings
import warnings
warnings.filterwarnings('ignore')

# Paths

In [ ]:
paths = fetchPaths(base, DATASET, '', False)
path_data = paths['data']

configs = load_json(paths['configuration'])
config = configs['configuration']
model_setup = configs['model_setup']
model_config = configs['model_config']
optim_name = list(config['optimizer'].keys())[0]
optim_args = config['optimizer'][optim_name]
scheduler_name = list(config['scheduler'].keys())[0]
scheduler_args = config['scheduler'][scheduler_name]

# GPU Device & Seed

In [ ]:
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [ ]:
device = get_device(GPU_NAME)

## Load model and settings

In [ ]:
# transformers
from models.transform import transform
tf_train = transform[DATASET.lower()]['train']
tf_test = transform[DATASET.lower()]['test']

In [ ]:
from models.mnist_model import MNIST_Model
from models.fashionmnist_model import FashionMNIST_CNN
from models.gtsrb_model import GTSRB_CNN

models = {
    'mnist': MNIST_Model,
    'fashionmnist': FashionMNIST_CNN,
    'gtsrb': GTSRB_CNN
}

model_ = models[DATASET.lower()]

### Configurations

In [ ]:
lhl = 30

# Load / Split / DataLoader

In [ ]:
feature_names = get_labels(DATASET)

train_data = get_dataset(DATASET, path_data, train=True, transform=tf_train)
test_data = get_dataset(DATASET, path_data, train=False, transform=tf_test)
len(train_data), len(test_data)

In [ ]:
trainloader = get_dataLoader(train_data, model_config['batch_size'], True)
testloader = get_dataLoader(test_data, model_config['batch_size'], False)

In [ ]:
tf_denormalize = T.Normalize(
    mean=[-m / s for m, s in zip(tf_train.transforms[-1].mean, tf_train.transforms[-1].std)],
    std=[1/s for s in tf_train.transforms[-1].std]
)

In [ ]:
show_images_loader(trainloader, feature_names=feature_names, transform=tf_denormalize)

# Helper Functions

In [ ]:
model_setup['last_hidden_neurons'] = lhl

model = model_(**model_setup).to(device)
model = torch.compile(model)
nn.DataParallel(model, device_ids=[CUDA])

In [ ]:
# loss function
loss_function = nn.CrossEntropyLoss()

# optimizer and scheduler
optimizer = getattr(torch.optim, optim_name)(model.parameters(), lr=model_config['lr'], **optim_args)
scheduler = getattr(torch.optim.lr_scheduler, scheduler_name)(optimizer, **scheduler_args)

In [ ]:
# # train 1 epoch for debuging
# model_config['epochs'] = 1

In [ ]:
# training testing attributes
kwargs = {
    'model': model,
    'loss_function': loss_function,
    'optimizer': optimizer,
    'lr_scheduler': scheduler,
    'device': device,
    'model_path': None, # to save the model pass a valid path
    'trainloader': trainloader,
    'testloader': testloader,
    'config': model_config
}

## Run Training

In [ ]:
# train
(train_losses, train_accs,
test_losses, test_accs,
train_loss, train_acc,
test_loss, test_acc,
confusion_matrix_train,
confusion_matrix_test,
model_path) = run_training_testing(**kwargs)

## Load Best Model and save LHL

In [ ]:
if model_path is not None:
    # load best model
    load_checkpoint(model, model_path)

    # normalize and save matrix
    confusion_matrix_test_norm = normalize_confusion_matrix(confusion_matrix_test)
    save_confusion_matrix(confusion_matrix_test_norm, path_saved_model, model_name, 'test')

    # export last hidden layer data
    export_last_hidden_layer(trainloader, model, device, lhl, None, path_lhl_raw, model_name, 'raw_train')
    export_last_hidden_layer(testloader, model, device, lhl, None, path_lhl_raw, model_name, 'raw_test')

## Export PCA

In [ ]:
if model_path is not None:
    # model postfix
    postfix = f"{optim_name}-{model_config['batch_size']}-{lhl}"
    model_name = f"{DATASET}_{postfix}"

    # get paths
    paths_ = fetchPaths(base, DATASET, postfix)
    p_lhl = paths_['lhl']
    p_lhl_raw = paths_['lhl_raw']
    p_lhl_pca = paths_['lhl_pca']

    # load data
    train = pd.read_csv(p_lhl_raw / f'{model_name}_raw_train.csv')
    test = pd.read_csv(p_lhl_raw / f'{model_name}_raw_test.csv')

    # fit scaler and pca
    # using only true instances from train
    true = train.loc[train['true'] == True]
    scaler_ = fitStandardScalerSingle(true, lhl)
    pca_ = fitPCASingle(true, scaler=scaler_, numNeurons=lhl)

    # save objects
    save_pickle(p_lhl / 'scaler.pkl', scaler_)
    save_pickle(p_lhl / 'pca.pkl', pca_)

    # transform and save data
    ## train
    applyPCASingle(p_lhl_pca, train, scaler=scaler_, numNeurons=lhl).to_csv(p_lhl_pca / f'{model_name}_pca_train.csv', index=False)

    ## test
    applyPCASingle(p_lhl_pca, test, scaler=scaler_, numNeurons=lhl).to_csv(p_lhl_pca / f'{model_name}_pca_test.csv', index=False)

    # save selected neurons
    gte_mean, top_third = neuronsLoadingsSingle(pca_, numNeurons=lhl, var_thld=0.9, loadings_thld=0.2)
    save_json(p_lhl / 'neurons_gte_mean.json', gte_mean)
    save_json(p_lhl / 'neurons_top_third.json', top_third)